In [1]:
import os
import altair as alt
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# alt.renderers.enable('html')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
folder_path = "Z:\data"

In [4]:
network_gdf = gpd.read_file(rf"{folder_path}\Delft_NL\network\edges\edges.geojson")
# panoids_gdf = gpd.read_file(rf"{folder_path}\Delft_NL\panoids\panoids.geojson")
coupling = pd.read_csv(os.path.join(os.pardir, "Data\coupling.csv"), index_col=0)

In [5]:
## Enter view section range
corner_1 = (51.979149531125806, 4.35470846552213)
corner_2 = (51.99066149868263, 4.366603737805931)

In [6]:
network_filtered = network_gdf.cx[corner_1[1]:corner_2[1], corner_1[0]:corner_2[0]].copy()
# network_filtered = network_gdf

In [7]:
network_filtered['panoids_present'] = ['No'] * len(network_filtered)
for section_index, section_data in network_filtered.iterrows():
    if coupling.loc[section_index, 'coupled_panoids'] is not np.NaN:
        network_filtered.loc[section_index, 'panoids_present'] = 'Yes'

In [8]:
## Make bogus category confidences
categories = ['category_1', 'category_2', 'category_3', 'category_4', 'category_5']
for category in categories:
    network_filtered[category] = [0] * len(network_filtered)

# TODO: Improve bogus generation so that total confidence is 1.0
for section_index, section in network_filtered.iterrows():
    if section['panoids_present'] == 'No':
        for _ in range(5):
            network_filtered.loc[section_index, category] = 0
    
    else:  # panoids present
        rand_vals = np.random.dirichlet(np.ones(len(categories)),size=1)[0]
        for rand_val_index, category in enumerate(categories):
            network_filtered.loc[section_index, category] = rand_vals[rand_val_index]


network_filtered['chosen_category'] = network_filtered[categories].idxmax(axis=1)
network_filtered.loc[network_filtered['panoids_present'] == 'No', 'chosen_category'] = 'no_category'
network_filtered.head(3)

,level_0,level_1,u,v,key,geometry,panoids_present,category_1,category_2,category_3,category_4,category_5,chosen_category
1009,8365,0,1519889790,8735050632,0,"LINESTRING (4.35990 51.98011, 4.35865 51.97990)",Yes,0.035075,0.279212,0.388368,0.140670,0.156675,category_3
1010,15715,0,8735050632,8735050637,0,"LINESTRING (4.35980 51.98056, 4.35982 51.98052...",No,0.000000,0.000000,0.000000,0.000000,0.000000,no_category
1013,8364,0,1519889790,8735025680,0,"LINESTRING (4.35854 51.98015, 4.35865 51.97990)",Yes,0.302267,0.233127,0.377510,0.026026,0.061071,category_3


In [9]:
network_filtered = network_filtered.reset_index()


### Two versions: 
1. Point select, view confidence when selecting
2. Area select, view types of category chosen when selecting

In [10]:
point = alt.selection_point(fields=['index'])
domain = ['no_category', 'category_1', 'category_2', 'category_3', 'category_4', 'category_5']
range_ = ['lightgray', 'green', 'red', 'blue', 'purple', 'orange']


roads = alt.Chart(network_filtered).mark_geoshape(
    tooltip=True,
    filled=False
).encode(
    tooltip=['index:N', 'chosen_category:N'],
    color = alt.condition(point,
                          alt.Color('chosen_category:N').scale(domain=domain, range=range_),
                          alt.value('lightgray')),
).properties(
    width=600,
    height=600
).project(
    'mercator'
).add_params(
    point
)

bars = alt.Chart(pd.melt(network_filtered,
                         id_vars=['index'],
                         value_vars=['category_1', 'category_2', 'category_3', 'category_4', 'category_5'])
                         ).mark_bar().encode(
    x='variable',
    y='value:Q',
    color='variable'
).properties(
    width=250,
    height=400
).transform_filter(
    point
)

In [11]:
roads | bars

alt.HConcatChart(...)